In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import os

In [2]:
from pymatgen.core import Structure
import os
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write
# Optional: if ASE is not installed, you need to install it
# pip install ase

# Define the input and output directories
cif_dir = "cif_files"       # directory containing CIF files
xyz_dir = "xyz_files"       # directory to store the XYZ files
os.makedirs(xyz_dir, exist_ok=True)

# Loop over all files in the cif_dir
for cif_file in os.listdir(cif_dir):
    if cif_file.endswith(".cif"):
        cif_path = os.path.join(cif_dir, cif_file)
        
        # Read structure from CIF
        structure = Structure.from_file(cif_path)
        
        # Define the output filename
        xyz_filename = os.path.join(xyz_dir, cif_file.replace(".cif", ".xyz"))
        
        # Convert to ASE object and write .xyz


        atoms = AseAtomsAdaptor.get_atoms(structure)
        write(xyz_filename, atoms)

        print(f"Converted {cif_file} to {xyz_filename}")


Converted Li.cif to xyz_files/Li.xyz
Converted CsCl.cif to xyz_files/CsCl.xyz
Converted SnS2.cif to xyz_files/SnS2.xyz
Converted LiI.cif to xyz_files/LiI.xyz
Converted LiH.cif to xyz_files/LiH.xyz
Converted CuI.cif to xyz_files/CuI.xyz
Converted CoO2.cif to xyz_files/CoO2.xyz


/opt/miniconda3/envs/stress/lib/python3.12/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


In [3]:
delta = 0.05
scales = [1 + i * delta for i in range(-6, 40)] 

In [4]:
import os

input_dir = "./xyz_files"
output_dir = "./scan"

os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(input_dir):
    if not file.endswith(".xyz"):
        continue

    filepath = os.path.join(input_dir, file)
    base_name = os.path.splitext(file)[0]
    output_xyz_path = os.path.join(output_dir, base_name + ".xyz")

    # Read original .xyz file
    with open(filepath) as f:
        lines = f.readlines()

    natoms = int(lines[0])
    coords = []
    symbols = []

    for line in lines[2:2+natoms]:
        print(line)
        parts = line.split()
        s, x, y, z = parts[:4]
        symbols.append(s)
        coords.append([float(x), float(y), float(z)])

    # Write scaled geometries to one .xyz file
    with open(output_xyz_path, "w") as out:
        for i, scale in enumerate(scales):
            out.write(f"{natoms}\n")
            out.write(f"scale = {scale:.5f}\n")
            for atom, (x, y, z) in zip(symbols, coords):
                new_x, new_y, new_z = x * scale, y * scale, z * scale
                out.write(f"{atom:2s}  {new_x:.10f}  {new_y:.10f}  {new_z:.10f}\n")

    print(f"✅ Written scan file: {output_xyz_path}")

print("🎉 All XYZ scan files generated successfully.")


Co       0.00000000       0.00000000      12.52177863       4.00000000

Co       1.38938403       0.80216124      20.86963105       4.00000000

Co       0.00000000       1.60432249       4.17392621       4.00000000

O        0.00000000       0.00000000       3.23793787      -2.00000000

O        0.00000000       0.00000000      21.80561939      -2.00000000

O        1.38938403       0.80216124      11.58579029      -2.00000000

O        1.38938403       0.80216124       5.10991455      -2.00000000

O        0.00000000       1.60432249      19.93364271      -2.00000000

O        0.00000000       1.60432249      13.45776697      -2.00000000

✅ Written scan file: ./scan/CoO2.xyz
Li       0.00000000       0.00000000       0.00000000       1.00000000

Li       2.98417741       2.98417741       0.00000000       1.00000000

Li       2.98417741       0.00000000       2.98417741       1.00000000

Li      -0.00000000       2.98417741       2.98417741       1.00000000

I        0.00000000       0

In [5]:
import re
def find_eq_cell(input_xyz):
    """
    Find the equilibrium cell length from an XYZ file.
    returns a 3x3 numpy array representing the lattice matrix.
    parameters:
        input_xyz (str): Path to the input XYZ file containing the lattice information.
    """
    with open(input_xyz, "r") as f:
        lines = f.readlines()
        comment_line = lines[1].strip()

        lattice_match = re.search(r'Lattice="([^"]+)"', comment_line)
        if not lattice_match:
            raise ValueError(f"Lattice not found in equilibrium file: {input_xyz}")

        lattice_values = list(map(float, lattice_match.group(1).split()))
        lattice_matrix = np.array(lattice_values).reshape((3, 3))
        # if values very small make it zero 
        lattice_matrix[np.abs(lattice_matrix) < 1e-5] = 0.0

        return lattice_matrix
        print(lattice_matrix)



In [6]:
find_eq_cell("xyz_files/Li.xyz")

array([[4.33366467, 0.        , 0.        ],
       [0.        , 4.33366467, 0.        ],
       [0.        , 0.        , 4.33366467]])

In [7]:
def parse_xyz_frames(lines,d3s_values,d3_values,lattice_matrix):
        """
        Parse XYZ file frames and extract cell lengths and scales.
        
        Parameters:
        - lines (list): Lines from XYZ file
        
        Returns:
        - list: List of [frame_idx, scale, x_length, y_length, z_length, d3s_value, d3_value]
        """
        i = 0
        frame_idx = 0
        cell_lengths = []
        while i < len(scales):
            try:
                scale = scales[i]

                cell_param_a = np.float64(lattice_matrix[0][0])
                cell_param = cell_param_a * scales[i]

                d3s_value = d3s_values[frame_idx] if frame_idx < len(d3s_values) else "NA"
                d3_value = d3_values[frame_idx] if frame_idx < len(d3_values) else "NA"
                
                print(frame_idx, scale, cell_param, d3s_value, d3_value)
                cell_lengths.append([frame_idx, scale, cell_param, d3s_value, d3_value])

                # i += natoms + 2
                i +=1
                frame_idx += 1

            except Exception as e:
                print(f"❌ Error at frame {frame_idx}: {e}")
                break
                
        return cell_lengths

In [8]:
import numpy as np
import csv

def extract_scan_data_to_csv(input_xyz, d3s_file, d3_file, output_csv):
    """
    Parse XYZ trajectory and D3/D3S energies to produce a CSV with frame cell lengths and energies.

    Parameters:
    - input_xyz (str): Path to the .xyz scan file.
    - d3s_file (str): Path to the D3S energy file.
    - d3_file (str): Path to the D3 energy file.
    - output_csv (str): Output CSV file path.
    """

    d3_values = []
    d3s_values = []
    
    # Read D3S values
    with open(d3s_file, "r") as f:
        d3s_values = [float(line.strip()) for line in f if line.strip()]

    # Read D3 values
    with open(d3_file, "r") as f:
        d3_values = [float(line.strip()) for line in f if line.strip()]

    # Prepare XYZ parsing
    with open(input_xyz, "r") as f:
        lines = f.readlines()

    lattice_matrix = find_eq_cell(input_xyz)
    cell_lengths = parse_xyz_frames(lines, d3s_values, d3_values, lattice_matrix)
    
    # Write to CSV
    with open(output_csv, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Frame", "Scale", "a_length", "D3s_Energy", "D3_Energy"])
        writer.writerows(cell_lengths)

    print(f"✅ CSV with D3 energy written to {output_csv}")


In [9]:
# extract_scan_data_to_csv(
#     input_xyz="xyz_files/Li.xyz",
#     d3s_file="../D3S_BJ_scan.txt",
#     d3_file="../D3_BJ_scan.txt",
#     output_csv="output/Li_scan.csv"
# )

In [10]:
import os
import glob

# Save current directory
curr_dir = os.getcwd()

scan_dir = os.path.join(curr_dir, "scan")
xyz_dir = os.path.join(curr_dir, "xyz_files")

print(f"Current directory: {curr_dir}")
output_dir = os.path.join(curr_dir, "output")
os.makedirs(output_dir, exist_ok=True)

# Find all .xyz files in scan/
scan_files = glob.glob(os.path.join(scan_dir, "*.xyz"))

print(f"Found {len(scan_files)} xyz files in {scan_dir}")
# Go up one level where Test_script.py is located
os.chdir("..")

# Run Test_script.py for each scan.xyz
for scan_file in scan_files:
    print(f"Running Test_script.py on {scan_file}")
    print(f'xyz_files/{os.path.basename(scan_file)}')
    os.system(f"python Test_script_o.py \"{scan_file}\"")

    output_csv_file = f"comps/output/{os.path.basename(scan_file).replace('.xyz', '.csv')}"
    extract_scan_data_to_csv(
        input_xyz=f'comps/xyz_files/{os.path.basename(scan_file)}',
        d3s_file="D3S_BJ_scan.txt",
        d3_file="D3_BJ_scan.txt",
        output_csv=output_csv_file
    )

# Return to original directory
os.chdir(curr_dir)

print("✅ All scans processed.")


Current directory: /Users/partha/Desktop/research/D3S/comps
Found 7 xyz files in /Users/partha/Desktop/research/D3S/comps/scan
Running Test_script.py on /Users/partha/Desktop/research/D3S/comps/scan/CoO2.xyz
xyz_files/CoO2.xyz
0 0.7 1.945137642 -0.0009460061778553834 -0.0009426855208441731
1 0.75 2.0840760449999998 -0.0006707731607955736 -0.0006707516406793837
2 0.8 2.2230144480000003 -0.0005478305182720294 -0.0005472639784420345
3 0.85 2.361952851 -0.0005388352738294271 -0.0005382859814474619
4 0.9 2.500891254 -0.0006270295958156006 -0.0006266062518980967
5 0.95 2.639829657 -0.0008057446588923225 -0.0008052421163339948
6 1.0 2.77876806 -0.0010614034199468224 -0.001060192172184217
7 1.05 2.917706463 -0.0013538715385419333 -0.0013503680236118568
8 1.1 3.056644866 -0.0016207365592691503 -0.0016111276169629258
9 1.15 3.1955832689999997 -0.0018365957492285155 -0.0018114540645454722
10 1.2 3.3345216719999997 -0.0019942111105843933 -0.0020206410272117573
11 1.25 3.473460075 -0.00204746989059